In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")


# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
class ShapesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "housematerial"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 4  # background + 3 shapes

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = 800
    IMAGE_MAX_DIM = 1024

    # Use smaller anchors because our image and objects are small
    #RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)
    
    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 50
    # TRAIN_ROIS_PER_IMAGE = 20

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 100

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = ShapesConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                17
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [3]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [4]:
class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.
Loading weights from  /home/jupyter/Mask_RCNN/logs/housematerial20210415T2000/mask_rcnn_housematerial_0040.h5
Re-starting from epoch 40


In [54]:
filenames=[]
original_images=[]
folder = "/home/jupyter/tutorials/storage/detect/downloads_10"
#folder = "/home/jupyter/Mask_RCNN/dataset/detect"
for filename in os.listdir(folder):
    img = cv2.imread(os.path.join(folder,filename))
    if img is not None:
        original_images.append(img)
        filenames.append(filename)

In [55]:
results = []
all_results = []
i = 1
for image in original_images:
    result = model.detect([image], verbose=0)
    r = result[0]
    results.append(r['class_ids'])
    all_results.append(r)
    print(i)
    i = i+1

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [56]:
statis = []
for all_result in all_results:
    dict = {}
    dict[1] = 0
    dict[2] = 0
    dict[3] = 0
    dict[4] = 0
    i = 0
    for class_id in all_result['class_ids']:
        dict[class_id] = dict.get(class_id,0) + all_result['masks'][:,:,i].reshape(1,-1).astype(np.int).sum()
        i = i+1
    statis.append(dict)
statis

[{1: 27962, 2: 0, 3: 0, 4: 0},
 {1: 15498, 2: 0, 3: 0, 4: 43308},
 {1: 126707, 2: 0, 3: 0, 4: 0},
 {1: 56238, 2: 3505, 3: 0, 4: 0},
 {1: 103641, 2: 0, 3: 0, 4: 0},
 {1: 19497, 2: 0, 3: 0, 4: 0},
 {1: 88163, 2: 0, 3: 25892, 4: 0},
 {1: 6885, 2: 0, 3: 0, 4: 0},
 {1: 81197, 2: 0, 3: 0, 4: 0},
 {1: 72324, 2: 0, 3: 0, 4: 0},
 {1: 25177, 2: 0, 3: 0, 4: 0},
 {1: 88115, 2: 0, 3: 0, 4: 68335},
 {1: 1414, 2: 0, 3: 0, 4: 0},
 {1: 26905, 2: 33416, 3: 0, 4: 0},
 {1: 33344, 2: 0, 3: 0, 4: 0},
 {1: 35036, 2: 0, 3: 0, 4: 0},
 {1: 122666, 2: 0, 3: 0, 4: 0},
 {1: 25258, 2: 18368, 3: 0, 4: 0},
 {1: 81439, 2: 0, 3: 0, 4: 0},
 {1: 32268, 2: 0, 3: 0, 4: 0},
 {1: 31195, 2: 0, 3: 0, 4: 0},
 {1: 10059, 2: 0, 3: 0, 4: 0},
 {1: 18885, 2: 0, 3: 41685, 4: 0},
 {1: 11550, 2: 0, 3: 71174, 4: 0},
 {1: 8095, 2: 0, 3: 0, 4: 0},
 {1: 115934, 2: 2666, 3: 0, 4: 0},
 {1: 31063, 2: 0, 3: 79448, 4: 0},
 {1: 158416, 2: 0, 3: 0, 4: 0},
 {1: 23993, 2: 0, 3: 0, 4: 78971},
 {1: 73211, 2: 0, 3: 0, 4: 0},
 {1: 127827, 2: 0, 3: 0, 4

In [57]:
import pandas as pd
statis = pd.DataFrame(statis)
statis

,1,2,3,4
0,27962,0,0,0
1,15498,0,0,43308
2,126707,0,0,0
3,56238,3505,0,0
4,103641,0,0,0
...,...,...,...,...
2378,29614,0,64272,0
2379,39373,0,0,0
2380,19923,0,36266,0
2381,52113,0,0,0


In [58]:
# dt_statis.to_excel('statis.xlsx')
statis['filename'] = filenames

In [59]:
statis.to_excel('area_ori.xlsx')
statis

,1,2,3,4,filename
0,27962,0,0,0,280.jpg
1,15498,0,0,43308,1612.jpg
2,126707,0,0,0,2106.jpg
3,56238,3505,0,0,2622.jpg
4,103641,0,0,0,2123.jpg
...,...,...,...,...,...
2378,29614,0,64272,0,43.jpg
2379,39373,0,0,0,3153.jpg
2380,19923,0,36266,0,811.jpg
2381,52113,0,0,0,2031.jpg


In [79]:
statis['filename'] = statis['filename'].map(lambda x: x.rstrip('.jpg'))

AttributeError: 'int' object has no attribute 'rstrip'

In [81]:
statis

,1,2,3,4,filename,Unnamed: 0,building_lat,building_lon,original_number
0,27962,0,0,0,280,247,51.515499,-0.101109,280
1,15498,0,0,43308,1612,1145,51.516266,-0.084683,1612
2,126707,0,0,0,2106,1414,51.511070,-0.084765,2106
3,56238,3505,0,0,2622,1808,51.513683,-0.102872,2622
4,103641,0,0,0,2123,1426,51.516354,-0.095523,2123
...,...,...,...,...,...,...,...,...,...
2367,29614,0,64272,0,43,35,51.512077,-0.083609,43
2368,39373,0,0,0,3153,2185,51.518459,-0.099007,3153
2369,19923,0,36266,0,811,625,51.517181,-0.074056,811
2370,52113,0,0,0,2031,1362,51.512180,-0.082621,2031


In [112]:
filtered

,building_lat,building_lon,original_number
0,51.510784,-0.091227,0
1,51.511627,-0.092403,1
2,51.514328,-0.091670,5
3,51.513775,-0.089974,6
4,51.513062,-0.087635,7
...,...,...,...
2381,51.515301,-0.096042,3454
2382,51.516991,-0.091228,3455
2383,51.515282,-0.095988,3456
2384,51.517033,-0.096512,3457


In [110]:
filtered = pd.read_excel('fitered_final.xlsx')

In [97]:
num_ori = pd.read_excel('num_ori.xlsx')

In [102]:
num_ori  = num_ori.reset_index()

In [100]:
area_ori = pd.read_excel('area_ori.xlsx')

In [105]:
area_ori   = area_ori.reset_index()

In [106]:
num_ori = num_ori.merge(area_ori, left_on='index', right_on='index')

In [108]:
num_ori['filename'] = num_ori['filename'].map(lambda x: x.rstrip('.jpg'))

In [109]:
num_ori['filename'] = num_ori['filename'].astype(int)

In [116]:
num_ori = num_ori.merge(filtered, left_on='filename', right_on='original_number')

In [125]:
aaaa = num_ori.dropna()

In [127]:
aaaa.to_excel("aaaa.xlsx")

In [126]:
len(aaaa)

2372

In [115]:
len(filtered)

2386

In [85]:
num_ori = num_ori.merge(statis, left_on='key', right_on='key')

KeyError: 'key'

In [ ]:
num_ori

In [93]:
number = pd.read_excel('final.xlsx')

In [94]:
number

,Unnamed: 0,building_lat,building_lon,original_number
0,0,51.510784,-0.091227,0
1,1,51.511627,-0.092403,1
2,2,51.514328,-0.091670,5
3,3,51.513775,-0.089974,6
4,4,51.513062,-0.087635,7
...,...,...,...,...
2381,2381,51.515301,-0.096042,3454
2382,2382,51.516991,-0.091228,3455
2383,2383,51.515282,-0.095988,3456
2384,2384,51.517033,-0.096512,3457


In [67]:
statis['filename'] = statis['filename'].astype(int)

In [71]:
statis = statis.merge(number, left_on='filename', right_on='original_number')

In [78]:
statis

,1,2,3,4,filename,Unnamed: 0,building_lat,building_lon,original_number
0,27962,0,0,0,280,247,51.515499,-0.101109,280
1,15498,0,0,43308,1612,1145,51.516266,-0.084683,1612
2,126707,0,0,0,2106,1414,51.511070,-0.084765,2106
3,56238,3505,0,0,2622,1808,51.513683,-0.102872,2622
4,103641,0,0,0,2123,1426,51.516354,-0.095523,2123
...,...,...,...,...,...,...,...,...,...
2367,29614,0,64272,0,43,35,51.512077,-0.083609,43
2368,39373,0,0,0,3153,2185,51.518459,-0.099007,3153
2369,19923,0,36266,0,811,625,51.517181,-0.074056,811
2370,52113,0,0,0,2031,1362,51.512180,-0.082621,2031


In [69]:
statis.to_excel('finalfinal.xlsx')

In [7]:
dt = all_results

In [13]:
all_results[0]['masks'].shape

(400, 400, 5)

In [14]:
all_results[1]['masks'].shape

(400, 400, 7)

In [19]:
all_results[1]['class_ids']

array([1, 1, 1, 1, 1, 2, 2], dtype=int32)

In [46]:
area = 0
for i in range(5):
    area = area + all_results[0]['masks'][:,:,i].reshape(1,-1).astype(np.int).sum()
print(area)

32406


In [47]:
all_results

[{'rois': array([[ 60, 219, 122, 270],
         [192, 358, 274, 395],
         [197, 218, 267, 271],
         [  0,  26, 262, 241],
         [  7, 281, 125, 359]], dtype=int32),
  'class_ids': array([1, 1, 1, 2, 2], dtype=int32),
  'scores': array([0.93466675, 0.91730905, 0.7793472 , 0.74394894, 0.73242295],
        dtype=float32),
  'masks': array([[[False, False, False, False, False],
          [False, False, False, False, False],
          [False, False, False, False, False],
          ...,
          [False, False, False, False, False],
          [False, False, False, False, False],
          [False, False, False, False, False]],
  
         [[False, False, False, False, False],
          [False, False, False, False, False],
          [False, False, False, False, False],
          ...,
          [False, False, False, False, False],
          [False, False, False, False, False],
          [False, False, False, False, False]],
  
         [[False, False, False, False, False],
        

In [48]:
statis = []
for all_result in all_results:
    dict = {}
    dict[1] = 0
    dict[2] = 0
    dict[3] = 0
    dict[4] = 0
    i = 0
    for class_id in all_result['class_ids']:
        dict[class_id] = dict.get(class_id,0) + all_result['masks'][:,:,i].reshape(1,-1).astype(np.int).sum()
        i = i+1
    statis.append(dict)
statis


[{1: 8933, 2: 23473, 3: 0, 4: 0}, {1: 7930, 2: 22088, 3: 0, 4: 0}]

In [ ]:
import pandas as pd
dt_statis = pd.DataFrame(statis)
dt_statis

In [49]:
all_results[0]['masks'][:,:,0].reshape(1,-1).astype(np.int).sum()

2737

In [50]:
all_results[0]['masks'][:,:,1].reshape(1,-1).astype(np.int).sum()

2782

In [51]:
all_results[0]['masks'][:,:,2].reshape(1,-1).astype(np.int).sum()

3414

In [52]:
all_results[0]['masks'][:,:,3].reshape(1,-1).astype(np.int).sum()

15690

In [53]:
all_results[0]['masks'][:,:,4].reshape(1,-1).astype(np.int).sum()

7783

In [4]:
a = {1:2,3:4,5:6}
b = {1:2,3:4,5:6}

In [8]:
c = [a,b]
type(c)
c

[{1: 2, 3: 4, 5: 6}, {1: 2, 3: 4, 5: 6}]

In [6]:
with open('data.json', 'w') as fp:
    json.dump(, fp)

In [ ]:
for resul in all_results:
    resul['rois'] = resul['rois'].tolist()
    resul['class_ids'] = resul['class_ids'].tolist()
    resul['scores'] = resul['scores'].tolist()
    resul['masks'] = resul['masks'].tolist()

In [1]:
import pandas as pd
statis = pd.read_excel('statis.xlsx')

In [2]:
statis

,Unnamed: 0,1,2,3,4,filename
0,0,8,0,0,0,280.jpg
1,1,5,0,0,1,1612.jpg
2,2,5,0,0,0,2106.jpg
3,3,9,1,0,0,2622.jpg
4,4,8,0,0,0,2123.jpg
...,...,...,...,...,...,...
2378,2378,8,0,1,0,43.jpg
2379,2379,8,0,0,0,3153.jpg
2380,2380,12,0,1,0,811.jpg
2381,2381,3,0,0,0,2031.jpg


In [4]:
statis['filename'] = statis['filename'].map(lambda x: x.rstrip('.jpg'))

In [6]:
statis

,Unnamed: 0,1,2,3,4,filename
0,0,8,0,0,0,280
1,1,5,0,0,1,1612
2,2,5,0,0,0,2106
3,3,9,1,0,0,2622
4,4,8,0,0,0,2123
...,...,...,...,...,...,...
2378,2378,8,0,1,0,43
2379,2379,8,0,0,0,3153
2380,2380,12,0,1,0,811
2381,2381,3,0,0,0,2031


In [5]:
origin = pd.read_excel('fitered_final.xlsx')

In [13]:
origin.merge(statis, left_on='original_number', right_on='filename')


,building_lat,building_lon,original_number,Unnamed: 0,1,2,3,4,filename
0,51.510784,-0.091227,0,1847,8,0,0,0,0
1,51.511627,-0.092403,1,560,19,0,0,0,1
2,51.514328,-0.091670,5,1273,10,1,0,0,5
3,51.513775,-0.089974,6,2090,12,0,1,0,6
4,51.513062,-0.087635,7,1325,4,0,0,0,7
...,...,...,...,...,...,...,...,...,...
2367,51.515461,-0.088859,3450,343,11,0,0,0,3450
2368,51.515301,-0.096042,3454,886,9,0,0,0,3454
2369,51.516991,-0.091228,3455,1482,5,0,0,0,3455
2370,51.515282,-0.095988,3456,1357,10,0,0,0,3456


In [9]:
type(origin['original_number'][0])

numpy.int64

In [12]:
type(statis['filename'][0])

numpy.int64

In [11]:
statis['filename'] = statis['filename'].astype(int)

In [14]:


origin.to_excel('final.xlsx')